In [ ]:
## add ignore warnings for now, will remove and debug once full algorithm is complete
# import warnings
# warnings.filterwarnings("ignore")

## import packages/libraries
from time import perf_counter, process_time, clock_gettime_ns, CLOCK_REALTIME
import numpy as np
import pandas as pd
from itertools import product
from multiprocessing import Pool, cpu_count
from PIL import Image
import sys
import sqlite3
import cv2 as cv

from cProfile import Profile
from pstats import SortKey, Stats
import psutil

## append filepath to allow files to be called from within project folder
sys.path.append('/home/gerard/Desktop/capstone_project/patoms')
sys.path.append('/home/gerard/Desktop/capstone_project')

## call locally created functions
from b_snapshot_2d_pattern_v9 import patoms2d
from c_pattern_2d_compare_v9 import pattern_compare_2d

from scipy.special import bernoulli, zeta

In [ ]:
# generate data (string of frames of a simple geometric shape)
image_path = 'red_circle.png'

image = Image.open(image_path)

image_array = np.array(image)[:,:,3]

top = 900
bottom = 10
left = 10
right = 900

padded_arrays = []
for i in range(0, 900, 6):
    pad_array = np.pad(image_array, ((top-i, bottom+i), (left+i, right-i)), 'constant', constant_values=(0)) # top, bottom, left, right
    padded_arrays.append(pad_array)


[ WARN:0@0.870] global cap_gstreamer.cpp:1173 isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


In [ ]:
source_db = "inmemory.db"  # Change this to your actual database file
memory_db = sqlite3.connect(":memory:")  # Create an in-memory DB

In [4]:
# Attach to disk database and copy all data into memory
disk_conn = sqlite3.connect(source_db)
disk_conn.backup(memory_db)  # Copies entire DB into memory
disk_conn.close()  # Close disk connection, work only in memory

In [5]:
cursor = memory_db.cursor()

In [6]:
# read 2d ref database into memory and convert to ref patoms for comparison
working_ref_patoms = []
ref_names = [name for (name,) in cursor.execute("select name from sqlite_master where type='table' and name like 'ref%';").fetchall()][:20]
for i in ref_names:
    table = cursor.execute(f"select * from {i};").fetchall()
    table_rows = []
    for row in table:
        table_row = [np.frombuffer(elem, dtype=np.float32) for elem in row]
        table_rows.append(table_row)
    table_array = np.array(table_rows)[:,:,-1]
    working_ref_patoms.append(table_array)

ref_patoms_array = np.vstack(working_ref_patoms).astype('float32')
ref_indices = np.unique(ref_patoms_array[:,7],axis=0)


In [7]:
# Function to measure total CPU time of all processes
def get_total_cpu_time():
    process = psutil.Process()  # Get the main process
    total_time = process.cpu_times().user  # Get CPU time of main process
    for child in process.children(recursive=True):  # Add child process CPU time
        total_time += child.cpu_times().user
    return total_time

In [ ]:
with Profile() as profile:
    start_perf = perf_counter()
    start_cpu = get_total_cpu_time()
    s = perf_counter(), process_time()
    val = 0
    while val < 1:
        # frame = np.random.randint(0, 256, (480, 640), dtype=np.uint8) # 307200 bytes
        # print(frame.nbytes)
        ret, frame = cap.read()
        frame = (frame[..., 0] << 16) | (frame[..., 1] << 8) | frame[..., 2]
        # print(frame.nbytes)
        frame = (frame - frame.min()) / (frame.max() - frame.min())
        x_len = frame.shape[0]
        y_len = frame.shape[1]
        ####################### FIRST TASK: FIND PATTERNS IN FRAME ######################
        frame_patoms = patoms2d(x_len, y_len, frame, val)
        # output: [colour, norm_x, norm_y, pattern_centroid_x, pattern_centroid_y, quad, quad_cnt, frame_ind_arr]
        #frame_patoms_arr = np.vstack((frame_patoms))
        num_patoms = len(frame_patoms)
        print(num_patoms)
        ############## SECOND TASK: COMPARE NEW PATOMS AGAINST REF PATOMS ###############
        atime = perf_counter(), process_time()
        start_cpu = get_total_cpu_time()
        with Pool(processes=8) as pool:
            indices = list(product(range(num_patoms), range(len(working_ref_patoms))))
            items = [(frame_patoms[i[0]], working_ref_patoms[i[1]]) for i in indices]
            comp_results = pool.starmap(pattern_compare_2d, items) #e.g. ['pcol','px','py','pxc','pyc','pq','pqlen','pfind','xc_d','yc_d','x_d','y_d','rfind','similar']
            end_cpu = get_total_cpu_time()
            print("Real Time to compare 2D patterns with multiprocessing (secs):", (perf_counter()-atime[0]))
            print("CPU Time to compare 2D patterns with multiprocessing (secs):", (process_time()-atime[1]))
            print(f"Total CPU Time (all processes): {end_cpu - start_cpu:.8f} sec")
            # loop through the output of the comparison function
            witime = perf_counter(), process_time()
            # for ix, i in enumerate(comp_results):
            #     table_data = np.unique(i[:,:-2], axis=0)
            #     #print(table_data)
            #     existing_tables = [names for (names,) in cursor.execute("select name from sqlite_master where type='table' and name not like 'ref%';").fetchall()]
            #     next_table_num = int(existing_tables.pop(-1)[-6:]) + 1 
            #     next_table_num = str(next_table_num).zfill(6)
            #     ref_patom_ind = i[:,12].astype('int32')
            #     ref_patom_ind = np.unique(ref_patom_ind).tolist() # contains list of all ref patoms used in comparison
            #     #extract reference tables that are similar to the patom in the current iteration
            #     match_patoms = i[i[:,13].min() >= 0.8]
                #print(match_patoms)
            #     if match_patoms.shape[0] > 0:
            #         for k in ref_patom_ind:
            #             table_num = ref_names[k][-6:]
            #             cursor.executemany(f"INSERT INTO nonref_{table_num}(colour, x_pos_dist, y_pos_dist, x_cent, y_cent, quad, quad_cnt, frame_ind, \
            #                                 xc_d, yc_d, x_d, y_d) VALUES (?,?,?,?,?,?,?,?,?,?,?,?)", table_data)
            #     # if no matching ref patoms, write new patom to new table
            #     else:
            #         cursor.execute(f"CREATE TABLE nonref_{next_table_num}(colour, x_pos_dist, y_pos_dist, x_cent, y_cent, quad, quad_cnt, frame_ind, \
            #                     xc_d, yc_d, x_d, y_d);")
            #         cursor.executemany(f"INSERT INTO nonref_{next_table_num}(colour, x_pos_dist, y_pos_dist, x_cent, y_cent, quad, quad_cnt, frame_ind, \
            #                         xc_d, yc_d, x_d, y_d) VALUES (?,?,?,?,?,?,?,?,?,?,?,?)", table_data) 
            #         cursor.execute(f"CREATE TABLE ref_{next_table_num}(colour, x_pos_dist, y_pos_dist, x_cent, y_cent, quad, quad_cnt, frame_ind, \
            #                     xc_d, yc_d, x_d, y_d);")
            #         cursor.executemany(f"INSERT INTO ref_{next_table_num}(colour, x_pos_dist, y_pos_dist, x_cent, y_cent, quad, quad_cnt, frame_ind, \
            #                         xc_d, yc_d, x_d, y_d) VALUES (?,?,?,?,?,?,?,?,?,?,?,?)", table_data)     
                
            # memory_db.commit() # have to include this commit newly created tables but this is a massive bottleneck - maybe create database in memory?    
            ## major bottleneck is the insert/create steps
            print("Real Time to write/insert patoms to existing/new tables in database (secs):", (perf_counter()-witime[0]))
            print("CPU Time to write/insert patoms to existing/new tables in database (secs):", (process_time()-witime[1]))
        val += 1
    
    
    print("Real Time to process 1 second of data against 10 reference patoms(secs):", (perf_counter()-s[0]))
    print("CPU Time to process 1 second of data against 10 reference patoms(secs):", (process_time()-s[1]))
    #print("Time to get patoms from 1 seconds of data (mins):", (perf_counter()-s)/60)

    end_perf = perf_counter()
    end_cpu = get_total_cpu_time()

    print(f"\nTotal Elapsed Time (perf_counter): {end_perf - start_perf:.2f} sec")
    print(f"Total CPU Time (all processes): {end_cpu - start_cpu:.2f} sec")

    (
        Stats(profile)
        .strip_dirs()
        .sort_stats(SortKey.CUMULATIVE)
        .print_stats()
    )

Real Time to get 2D patterns with multiprocessing (secs): 2.704749960999834
CPU Time to get 2D patterns with multiprocessing (secs): 2.710324203
108


In [ ]:
########################################################################################################
############## next steps is reconcile ref patoms and put time order on patom sequence #################
########################################################################################################

# fileds in tables: (colour, x_pos_dist, y_pos_dist, x_cent, y_cent, quad, quad_cnt, frame_ind, xc_d, yc_d, x_d, y_d)
patom_sequences = [[] for _ in range(900)]
table_data = []
nonref_names = [name for (name,) in cursor.execute("select name from sqlite_master where type='table' and name not like 'ref%';").fetchall()]; print(len(nonref_names))
ref_names = [name for (name,) in cursor.execute("select name from sqlite_master where type='table' and name like 'ref%';").fetchall()]; print(len(ref_names))
for name in nonref_names:
    row = cursor.execute(f"select \
                               colour, x_pos_dist, y_pos_dist, x_cent, y_cent, segment, segment_cnt, sequence_ind, col_d, xc_d, yc_d, x_d, y_d \
                               from {name} order by segment, x_pos_dist, y_pos_dist;").fetchall()
    table_data.append(row)
    # seq_ind = cursor.execute(f"select cast(avg(frame_ind) as integer) as seq_ind, min(frame_ind) as min_seq_ind, max(frame_ind) as max_seq_ind from {name};").fetchall()
    # print(seq_ind)
        # patom_sequences[seq_ind].append(name[3:])


7936
7936


In [ ]:
for x in table_data[0]:
    print(x)

(0.0, -1.0, 0.8360128402709961, 0.1841689646244049, 0.08407414704561234, 0.0, 48.0, 48.0, 0.0, 0.0, 0.0, 0.0, 0.0)
(0.0, -0.9463087320327759, -0.9003215432167053, 0.1841689646244049, 0.08407414704561234, 0.0, 48.0, 48.0, 0.0, 0.0, 0.0, 0.0, 0.0)
(0.0, -0.9194630980491638, -1.0, 0.1841689646244049, 0.08407414704561234, 0.0, 48.0, 48.0, 0.0, 0.0, 0.0, 0.0, 0.0)
(0.0, -0.7852349281311035, -0.022508038207888603, 0.1841689646244049, 0.08407414704561234, 0.0, 48.0, 48.0, 0.0, 0.0, 0.0, 0.0, 0.0)
(0.0, -0.753915011882782, -0.8167202472686768, 0.1841689646244049, 0.08407414704561234, 0.0, 48.0, 48.0, 0.0, 0.0, 0.0, 0.0, 0.0)
(0.0, -0.6689038276672363, -0.03215434029698372, 0.1841689646244049, 0.08407414704561234, 0.0, 48.0, 48.0, 0.0, 0.0, 0.0, 0.0, 0.0)
(0.0, -0.5436241626739502, -0.9774919748306274, 0.1841689646244049, 0.08407414704561234, 0.0, 48.0, 48.0, 0.0, 0.0, 0.0, 0.0, 0.0)
(0.0, -0.5391498804092407, -0.9774919748306274, 0.1841689646244049, 0.08407414704561234, 0.0, 48.0, 48.0, 0.0, 0

In [ ]:
# memory_db.commit()

# # 5️⃣ Write the in-memory database back to a file
# dest_db = "my_new_database.db"  # New file to save changes
# new_disk_conn = sqlite3.connect(dest_db)
# memory_db.backup(new_disk_conn)  # Save in-memory DB to file
# new_disk_conn.close()

# # ✅ Done! You now have `my_new_database.db` with the new tables & data.
# memory_db.close()